<a href="https://www.kaggle.com/code/guitaristboy/transformer-from-scratch?scriptVersionId=209073102" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/input-txt/input.txt


In [37]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [38]:
# hyperparameter
# how many independent sequences will we process in parallel?
batch_size = 32
# what is the maximum context ellgnth for predeictions?
block_size = 8 
max_iters = 100000
eval_interval = 500
learning_rate = 1e-2
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 32

In [39]:
torch.manual_seed(1337)

In [5]:
with open('/kaggle/input/input-txt/input.txt','r',encoding='utf-8') as  f:
    text = f.read()

In [6]:
print('Length of the dataset in character is ' , len(text))

Length of the dataset in character is  1115394


In [7]:
# here are the unique character that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# create a mapping from characters to integers
stoi = {ch:i for i , ch in enumerate(chars)}
itos = {i:ch for i , ch in enumerate(chars)}
# encoder: take string , output a list of integers
encode = lambda s : [stoi[c] for c in s] 
# decoder: takes input of integers , outputs string
decode = lambda l : ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:

data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# lets now split up the data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [16]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb , yb = get_batch('train')
print('inputs :')
print(xb.shape)
print(xb)
print('targets : ')

print(yb.shape)
print(yb)


print('-------------')

# batch dimension
for b in range(batch_size):
    # time dimension
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target: {target}")

inputs :
torch.Size([32, 8])
tensor([[52, 57,  8,  0, 21,  5, 50, 50],
        [47, 58, 46,  1, 63, 53, 59,  6],
        [ 1, 53, 40, 43, 63, 12,  0, 26],
        [50, 42, 57, 58,  6,  0, 32, 46],
        [ 1, 21,  1, 51, 39, 63,  7,  7],
        [ 1, 59, 54,  1, 40, 63,  1, 19],
        [43,  1, 43, 52, 53, 59, 45, 46],
        [42, 10,  1, 61, 43,  1, 41, 39],
        [20, 53, 61,  1, 44, 53, 52, 42],
        [58, 43, 56, 51, 47, 52, 43, 57],
        [43,  1, 53, 44,  1, 58, 56, 59],
        [43, 52,  1, 44, 56, 53, 51,  1],
        [11,  1, 47, 58,  1, 47, 57,  1],
        [56, 47, 58,  1, 61, 43, 56, 43],
        [47, 43, 58,  6,  1, 53, 52,  1],
        [40, 56, 53, 58, 46, 43, 56,  1],
        [59, 56,  1, 53, 44,  1, 58, 46],
        [13, 52, 42,  1, 57, 39, 47, 42],
        [45, 39, 56, 50, 39, 52, 42,  8],
        [43, 39, 56, 57,  6,  0, 13, 52],
        [ 8,  0,  0, 17, 16, 35, 13, 30],
        [47, 57,  1, 58, 46, 43,  1, 61],
        [ 1, 52, 47, 43, 41, 43,  1, 28],
     

In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train" , "eval"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X , Y = get_batch(split)
            logits , loss = model(X , Y)
            losses[k] = loss.item()

        out[split] = losses.mean()

    model.train()
    return out

In [18]:
class Head(nn.Module):
    """ One head of self-attention"""

    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) #(B,T,C)
        q = self.query(x) #(B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,C) @ (B, C, T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T,:T] == 0 , float('-inf')) #(B,T,T)
        wei = F.softmax(wei , dim = -1) # (B,T,T)
        # perform the weighted aggregation of the values
        v = self.value(x) #(B,T,C)
        out = wei @ v #(B,T,T) @ (B,T,C) -> (B,T,C)
        return out

In [19]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_head , head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_head)])

    def forward(self,x):
        return torch.cat([h(x) for h in self.heads], dim = -1)

In [20]:
class FeedForward(nn.Module):
    """ a simple linear layer followed  by a non-linearity """

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,n_embd),
            nn.ReLU(),
        )

    def forward(self,x):
        return self.net(x)

In [29]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    
    def __init__(self,n_embd , n_head):
        # n_embd: embedding dimension , n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)

    def forward(self,x):
        x = self.sa(x)
        x = self.ffwd(x)
        return x

In [30]:
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.postion_embedding_table = nn.Embedding(block_size , n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd,n_head=4),
            Block(n_embd,n_head=4),
            Block(n_embd,n_head=4),
        )
        # self.sa_heads = MultiHeadAttention(4,n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
        # self.ffwd = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)
        
    def forward(self, idx , targets=None):
        B , T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) #(B , T , C)
        pos_emb = self.postion_embedding_table(torch.arange(T,device=device)) #(T,C)
        x = tok_emb + pos_emb # (B , T , C)
        # x = self.sa_heads(x) # applying multiple head of self-attention. (B,T,C)
        # x = self.ffwd(x) # (B,T,C)
        x = self.blocks(x) # (B,T,C) 
        logits = self.lm_head(x) #(B , T , vocab_size)
        
        if targets is None:
            loss = None
        else:
            B , T , C = logits.shape
            logits = logits.view(B * T , C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits,targets)
        return logits , loss

    def generate(self,idx,max_new_tokens):
        # ids is (B,T) array of indices in the current context
        for _ in  range(max_new_tokens):
            #crop idx to the last block_size tokens
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits , loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:,-1,:] # becomes(B,C)
            # apply softmax to get probabilites
            probs = F.softmax(logits,dim =-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs,num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx,idx_next), dim = 1) # (B , T+1)
        return idx

In [31]:
model = BigramLanguageModel()
m = model.to(device)
# logits , loss = m(xb,yb)
# print(logits.shape)
# print(loss)

In [32]:
# # create a PyTorch optimizer
# optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)

# for steps in range(10000):

#     # sample a batch of data
#     xb , yb = get_batch("train")

#     # evaluate the loss
#     logits , loss = m(xb,yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

# print(loss.item())

In [33]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(),lr = learning_rate)

In [34]:
for iter in range(max_iters):
    # every once in a while evalutate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f},val loss{losses['eval']:.4f}")

        # sample a batch of data
        xb , yb = get_batch('train')

        # evaluate the loss
        logits , loss = m(xb , yb)
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

# generate from teh model
context = torch.zeros((1,1),dtype = torch.long,device=device)

step 0: train loss 4.1556,val loss4.1563
step 500: train loss 4.1051,val loss4.1067
step 1000: train loss 3.9878,val loss3.9952
step 1500: train loss 3.7516,val loss3.7682
step 2000: train loss 3.7749,val loss3.8367
step 2500: train loss 3.5196,val loss3.5518
step 3000: train loss 3.5042,val loss3.5315
step 3500: train loss 3.4672,val loss3.4942
step 4000: train loss 3.4103,val loss3.4325
step 4500: train loss 3.3914,val loss3.4563
step 5000: train loss 3.3958,val loss3.4318
step 5500: train loss 3.3685,val loss3.4394
step 6000: train loss 3.3532,val loss3.4077
step 6500: train loss 3.3603,val loss3.3893
step 7000: train loss 3.3455,val loss3.3854
step 7500: train loss 3.3489,val loss3.3995
step 8000: train loss 3.3461,val loss3.3865
step 8500: train loss 3.3522,val loss3.3776
step 9000: train loss 3.3413,val loss3.3804
step 9500: train loss 3.3475,val loss3.3747
step 10000: train loss 3.3351,val loss3.3775
step 10500: train loss 3.3385,val loss3.3747
step 11000: train loss 3.3286,val 

In [35]:
print(decode(m.generate(torch.zeros((1,1),dtype=torch.long),max_new_tokens=500)[0].tolist()))


Waitan,nit tenrp n
C tetahhanndb nteloriu  nr
Oztt nw aD:
EAygap ytonnum fbomeonnt.
WnSDIFert,dit,utinn ngyisdnl tigo?AOU,scpew.
Paeuptot notano adisrdll  e  r dynknednesrs ti cisut tkneri kg rdrrb s,ygnnp'my
H
U MetyeIRRBEI
p
-AO
YPDNfu::
:YZDLILC:
IYA EN
N:U:UTOGSuoir nhoI

No g rr hooesygml:  c he nw,vlit.
SWonsn.Ooe eow,.
S  
Ute;kvot ,s ncegoen o'!
suvpunnlm ednw tyew m,usn d . AO
Ruti doerimaLAr  :KHA
oeetemy ':
s:'tnrtheCtot , cht cp ?
ISIsbse
Comtthif yofmenons tyrs Tetd eto rtee oleg er


## The mathematical trick in self-attention

In [21]:
# consider the following toy example:

torch.manual_seed(1337)
# batch , time , channels
B , T , C = 4 , 8 , 2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [22]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))

for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev , 0)

In [23]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [24]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [25]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print("--")
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])
